In [3]:
# imports
import pandas as pd

In [4]:
# TODO: Justify assumptions
MIN_LEN = 20
MAX_LEN = 200

LANG = ['deu', 'eng', 'fra']

DATA_SIZE = 5000
TEST_SIZE = 0.2

data = pd.read_csv('sentences.csv',
                  sep='\t',
                  encoding='utf8',
                  index_col=0,
                  names=['lang', 'text'])
print(data)



        lang                                 text
1        cmn                               我們試試看！
2        cmn                              我该去睡觉了。
3        cmn                              你在干什麼啊？
4        cmn                               這是什麼啊？
5        cmn               今天是６月１８号，也是Muiriel的生日！
...      ...                                  ...
9621308  eng                He is the one I need.
9621309  hun                        Nekem ő kell!
9621310  eng  She's the one I've ever dreamed of.
9621311  hun     Ő az, akiről mindig is álmodtam.
9621312  hun     Miért nem vetted fel a telefont?

[9184604 rows x 2 columns]


In [ ]:
# Filter text by length
filter_len = [True if MIN_LEN <= len(t) <= MAX_LEN else False for t in data['text']]
data = data[filter_len]

# Filter text by language
filter_lang = [True if l in LANG else False for l in data['lang']]
data = data[filter_lang]

In [ ]:
# Shuffle and crop data
data_sample = data.sample(n=DATA_SIZE)

# Split data into test set and training set
offset = int(TEST_SIZE * DATA_SIZE)
data_test = data_sample[:offset]
data_train = data_sample[offset:]
print(len(data_test))